In [15]:
import os
import sys
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)
input_path = PROJECT_ROOT + '/data/customers_raw.csv'
from src.part1.dataprofiler import DataProfiler
from src.utils.logger_config import setup_pipeline_logger

import pandas as pd

# Initialize our specialized logger
logger = setup_pipeline_logger()

logger.info("Pipeline Logger initialized. Ready for Data Quality Analysis.")

# Verify the data directory exists
if not os.path.exists('../data'):
    logger.error("Data directory not found!")
else:
    logger.info("Data directory verified.")

INFO: Pipeline Logger initialized. Ready for Data Quality Analysis.
INFO: Data directory verified.


part 1

In [19]:
# Instantiate and run
profiler = DataProfiler(input_path=input_path)
profiler.run_full_analysis(output_report_path='../src/part1/report/data_quality_report.txt')

INFO: Loaded 10 rows from customers_raw.csv
INFO: Analysis complete. Report saved to: ../src/part1/report/data_quality_report.txt
